# Homework 2 - CV Verification System

An agentic AI system that verifies CV claims against LinkedIn and Facebook profiles via MCP server tools.

# Set up

## Installing packages

In [2]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 34.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9


## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.


In [3]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

In [4]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 36.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 8.55MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 7.53MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 7.22MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 4.66MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

## Load and parse all CVs

In [5]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated across teams to support large-scale platforms.

• Applied analytical and problem-solving skills in production systems.

Education

McGill University

Bachelor of Science (BSc) in Marketing

Skills

Content Creation

SEO

Social Media

Graduated 2009

1





📄 CV_2.pdf
Minh Pham
Design Professional

Beijing, China | Hong Kong

Professional Experience

Manager, BCG

2022 – Present

• Led cross-functional teams on client-facing design initiatives.

• Managed project timelines, deliverables, and stakeholder communication.

• Applied design thinking to business and strategy problems.

Analyst, Tencent

2013 – 2017

• Conducted market and product analysis to support decision-making.

• Collaborated with design and engineering teams.

• Produced reports and insights for senior leadersh

# Connect to MCP Server and Initialize Agent

In [6]:
import asyncio
import json
import re
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# ---------------------------
# 1. Connect to MCP server
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools

# Build a tool name -> tool object mapping for execution
tool_map = {tool.name: tool for tool in tools}

print("Available tools:")
for t in tools:
    print(f"  - {t.name}")

# ---------------------------
# 2. Initialize LLM
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    vertexai=True,
    api_key=GEMINI_VERTEX_API_KEY,
    temperature=0,
)

# Alternatively use DeepSeek:
# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

Available tools:
  - search_facebook_users
  - get_facebook_profile
  - get_facebook_mutual_friends
  - search_linkedin_people
  - get_linkedin_profile
  - get_linkedin_interactions


# CV Verification Agent

## Architecture

The agent follows this workflow for each CV:

1. **CV Parsing**: Extract name, jobs, education, skills, location from CV text
2. **LinkedIn Search & Profile Retrieval**: Search for the candidate on LinkedIn, get full profile
3. **Facebook Search & Profile Retrieval**: Search for the candidate on Facebook, get full profile
4. **Cross-Verification**: Compare CV claims against LinkedIn/Facebook data
5. **Discrepancy Detection & Scoring**: Identify mismatches, generate a reliability score

The agent uses a **ReAct-style loop** with tool calling — the LLM decides which tools to call, receives results, and reasons about them iteratively.

In [7]:
# =====================================================
#  ReAct Agent Loop - executes tool calls iteratively
# =====================================================

async def run_agent(messages, llm_with_tools, tool_map, max_iterations=15):
    """
    Run a ReAct-style agent loop.
    The LLM generates tool calls, we execute them, feed results back,
    and repeat until the LLM produces a final text response (no tool calls).
    """
    for i in range(max_iterations):
        # Get LLM response
        response = await llm_with_tools.ainvoke(messages)
        messages.append(response)

        # If no tool calls, we're done
        if not response.tool_calls:
            return response.content, messages

        # Execute each tool call
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            tool_id = tool_call["id"]

            print(f"  🔧 Calling: {tool_name}({tool_args})")

            if tool_name in tool_map:
                try:
                    result = await tool_map[tool_name].ainvoke(tool_args)
                    # Extract text from MCP response
                    if isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict):
                        result_text = result[0].get("text", str(result))
                    else:
                        result_text = str(result)
                except Exception as e:
                    result_text = f"Error calling {tool_name}: {str(e)}"
            else:
                result_text = f"Tool '{tool_name}' not found."

            print(f"  📋 Result: {result_text[:200]}..." if len(str(result_text)) > 200 else f"  📋 Result: {result_text}")

            messages.append(ToolMessage(
                content=result_text,
                tool_call_id=tool_id
            ))

    return "Max iterations reached.", messages

print("Agent loop defined.")

Agent loop defined.


In [8]:
SYSTEM_PROMPT = """You are an expert CV/Resume verification agent. Your task is to verify the claims
made in a candidate's CV by cross-referencing with their LinkedIn and Facebook profiles.

You have access to the following tools:
1. search_facebook_users(q, limit, fuzzy) - Search for Facebook users by name
2. get_facebook_profile(user_id) - Get a Facebook user's full profile
3. get_facebook_mutual_friends(user_id_1, user_id_2) - Find mutual friends
4. search_linkedin_people(q, location, industry, limit, fuzzy) - Search LinkedIn profiles
5. get_linkedin_profile(person_id) - Get a LinkedIn user's full profile
6. get_linkedin_interactions(person_id) - Get LinkedIn engagement data

## Verification Workflow

For each CV, follow these steps:

### Step 1: Extract CV Information
Parse and identify:
- Full name
- Location (Prioritize using the first location)
- Industry mentioned(Usually before Professional)
- Job titles, companies, and date ranges
- Education (degree, school, year)
- Skills listed
- Any other notable claims

### Step 2: Search LinkedIn
- Use search_linkedin_people with args: the candidate's name and industry, set limit to 5
- Remember don't use the location arg in search_linkedin_people
- Get the full LinkedIn profile with get_linkedin_profile
- Try all the possible matched LinkedIn ids

### Step 3: Search Facebook
- Use search_facebook_users with the candidate's name, set limit to 5
- Get the full Facebook profile with get_facebook_profile

### Step 4: Cross-Verify and Detect Discrepancies
Compare CV claims against social media data. Check for:
- **Name mismatches** between CV, LinkedIn, and Facebook
- **Job title / company mismatches** (different title, different company, or jobs not listed)
- **Date inconsistencies** (overlapping employment periods, future dates, timeline gaps)
- **Education mismatches** (different degree, different school, different graduation year, field mismatch)
- **Location inconsistencies** between CV and profiles
- **Skill exaggeration** (skills listed on CV but low proficiency on LinkedIn)
- **Seniority inflation** (claiming senior roles when LinkedIn shows junior)
- **Logical impossibilities** (e.g., working before graduating, end date before start date, future employment dates)
- **Overlapping full-time roles** that seem implausible

### Step 5: Generate Final Assessment
After all verification steps, provide your assessment in EXACTLY this format:

```
DISCREPANCIES FOUND:
1. [Description of discrepancy]
2. [Description of discrepancy]
...

VERIFICATION SUMMARY:
[Brief overall assessment]

RELIABILITY_SCORE: [a float between 0.0 and 1.0]
```

Scoring Guidelines:
- **0.8-1.0:** CV is highly consistent with both LinkedIn and Facebook profiles. Minimal or no discrepancies.
- **0.51-0.79:** CV shows acceptable alignment in core education and professional experience (e.g., names of institutions, companies, and roles) with *at least one* social media profile (LinkedIn or Facebook). Minor discrepancies such as specific employment dates, "current" status indicators, or slight location variations are tolerated if the overall narrative holds true. If there are general matches for school and experience on LinkedIn, score should be 0.5 or above.
- **0.0-0.49:** CV presents severe and irreconcilable contradictions with *both* LinkedIn and Facebook profiles regarding fundamental claims like education or work history. This indicates significant misrepresentation or completely incorrect information.

IMPORTANT:
- Always search BOTH LinkedIn AND Facebook for each candidate
- Always retrieve full profiles (not just search results)
- Be thorough - check every claim that can be verified
- The final line of your response MUST contain RELIABILITY_SCORE: followed by a number
"""

print("System prompt defined.")

System prompt defined.


In [9]:
# =====================================================
#  Verify a single CV using the agent
# =====================================================

async def verify_single_cv(cv_text, cv_name, llm_with_tools, tool_map):
    """
    Verify a single CV by running the agent loop.
    Returns (score, report).
    """
    print(f"\n{'='*80}")
    print(f"🔍 Verifying: {cv_name}")
    print(f"{'='*80}")

    user_message = f"""Please verify the following CV. Follow the complete verification workflow:
search LinkedIn, search Facebook, retrieve full profiles, and cross-check all claims.

=== CV CONTENT ({cv_name}) ===
{cv_text}
=== END CV ===

Perform thorough verification and provide your RELIABILITY_SCORE at the end."""

    messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=user_message)
    ]

    final_response, messages = await run_agent(messages, llm_with_tools, tool_map, max_iterations=20)

    # Extract the reliability score from the response
    score = extract_score(final_response)

    print(f"\n📊 {cv_name} -> Score: {score}")
    print(f"{'='*80}\n")

    return score, final_response


def extract_score(response_text):
    """
    Extract RELIABILITY_SCORE from the agent's final response.
    Falls back to 0.5 if not found.
    """
    if not response_text:
        return 0.5

    # Ensure response_text is a string for regex searching
    if isinstance(response_text, list):
        # Assuming the list contains elements that can be stringified
        response_text = " ".join([str(item) for item in response_text])
    elif not isinstance(response_text, str):
        response_text = str(response_text)


    # Try to find RELIABILITY_SCORE: X.XX pattern
    patterns = [
        r'RELIABILITY_SCORE\s*:\s*([0-9]*\.?[0-9]+)',
        r'reliability_score\s*:\s*([0-9]*\.?[0-9]+)',
        r'Reliability Score\s*:\s*([0-9]*\.?[0-9]+)',
        r'score\s*:\s*([0-9]*\.?[0-9]+)',
    ]

    for pattern in patterns:
        match = re.search(pattern, response_text, re.IGNORECASE)
        if match:
            score = float(match.group(1))
            return max(0.0, min(1.0, score))  # Clamp to [0, 1]

    print("  ⚠️ Could not extract score, defaulting to 0.5")
    return 0.5


print("Verification functions defined.")

Verification functions defined.


# Run Verification on All 5 CVs

In [ ]:
await tools[0].ainvoke({'q': "John Smith", 'limit': 5})

[{'type': 'text',
  'text': '[{"id":8,"display_name":"John Smith","city":"Kowloon","country":"Hong Kong","match_type":"exact"},{"id":51,"display_name":"John Smith","city":"Central","country":"Hong Kong","match_type":"exact"},{"id":213,"display_name":"John Smith","city":"Singapore","country":"Singapore","match_type":"exact"},{"id":254,"display_name":"John Smith","city":"Causeway Bay","country":"Hong Kong","match_type":"exact"},{"id":322,"display_name":"John Smith","city":"Munich","country":"Germany","match_type":"exact"}]',
  'id': 'lc_34451600-14fc-4374-9f23-38e6f0153897'}]

In [ ]:
await tools[1].ainvoke({'user_id': 213})

[{'type': 'text',
  'text': '{"id":213,"display_name":"John Smith","original_name":"John Smith","city":"Singapore","country":"Singapore","hometown":"Hong Kong","bio":"Evidence ago much.  Amount travel find consumer door near president","status":"Engaged","education":"Bachelor\'s Degree","current_job":"Scientist","current_company":"Traveloka","interests":"Machine Learning, Reading, Cooking","friend_count":34,"friends":[205,471,9093,2344,247,1738,4778,9391,283,368,5105,3283,4022,6903,5177,1755,5628,5629,1075,1636,2271,3616,4956,5196,5592,5836,7124,7201,8027,8355,8577,8733,9176,9770],"posts":[{"id":1199,"content":"New personal best in Machine Learning! Progress feels amazing 💪"},{"id":1200,"content":"Late night ramen cravings = satisfied 🌮"},{"id":1201,"content":"Celebrating small wins today! 🎊"},{"id":1202,"content":"Family dinner never gets old ❤️ #blessed"},{"id":1203,"content":"Cousins gathering! Haven\'t seen everyone in ages! 👨\u200d👩\u200d👧\u200d👦"}]}',
  'id': 'lc_9ae36478-4ca9-46

In [ ]:
await tools[4].ainvoke({'person_id': 47})

[{'type': 'text',
  'text': '{"id":47,"name":"Minh Pham","headline":"Design Professional","city":"Beijing","country":"China","industry":"Design","status":"open_to_work","years_experience":12,"summary":"Choose reveal second allow evening authority. Difficult official already build onto. Between measure born feel reach. Subject later generation when.","skills":[{"name":"UI/UX","proficiency":1},{"name":"Prototyping","proficiency":5},{"name":"Graphic Design","proficiency":1}],"experience":[{"company":"BCG","title":"Manager","seniority":"junior","start_year":2022,"end_year":null,"is_current":true},{"company":"Tencent","title":"Analyst","seniority":"junior","start_year":2013,"end_year":2017,"is_current":false}],"education":[{"school":"The University of Hong Kong","degree":"BSc","field":"Design","start_year":2007,"end_year":2011}]}',
  'id': 'lc_b26fbdd5-fbaf-481e-94e3-6b8e040291da'}]

In [ ]:
await tools[3].ainvoke({'q': "Minh Pham", 'location': "China", 'limit': 5})

[{'type': 'text',
  'text': '[{"id":47,"name":"Minh Pham","headline":"Design Professional","industry":"Design","location":"Beijing, China","years_experience":12,"match_type":"exact"},{"id":70,"name":"Minh Pham","headline":"AI Professional","industry":"AI","location":"Beijing, China","years_experience":6,"match_type":"exact"},{"id":263,"name":"Minh Pham","headline":"Education Professional","industry":"Education","location":"Shenzhen, China","years_experience":6,"match_type":"exact"},{"id":465,"name":"Minh Pham","headline":"Software Professional","industry":"Software","location":"Shanghai, China","years_experience":4,"match_type":"exact"},{"id":973,"name":"Minh Pham","headline":"Marketing Professional","industry":"Marketing","location":"Beijing, China","years_experience":16,"match_type":"exact"}]',
  'id': 'lc_0743a132-a83c-4141-97d9-b3ca0e446c82'}]

In [ ]:
await tools[3].ainvoke({'q': 'John Smith', 'location': 'Singapore', 'industry': 'Marketing', 'limit': 5})

[{'type': 'text',
  'text': '[{"id":9,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":7,"match_type":"exact"},{"id":377,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":1,"match_type":"exact"},{"id":1636,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":17,"match_type":"exact"},{"id":5145,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":0,"match_type":"exact"},{"id":6151,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":10,"match_type":"exact"}]',
  'id': 'lc_48072ddf-8b85-4ffa-a7ed-ef4e795579c8'}]

In [10]:
# =====================================================
#  Run verification on all CVs
# =====================================================

scores = []
reports = []

for cv in all_cvs:
    score, report = await verify_single_cv(
        cv_text=cv["text"],
        cv_name=cv["file"],
        llm_with_tools=llm_with_tools,
        tool_map=tool_map
    )
    scores.append(score)
    reports.append(report)

print("\n" + "=" * 80)
print("FINAL SCORES:")
print("=" * 80)
for cv, s in zip(all_cvs, scores):
    status = "✅ VALID" if s > 0.5 else "❌ FLAGGED"
    print(f"  {cv['file']}: {s:.2f} -> {status}")
print(f"\nscores = {scores}")


🔍 Verifying: CV_1.pdf
  🔧 Calling: search_linkedin_people({'industry': 'Marketing', 'q': 'John Smith', 'limit': 5})
  📋 Result: [{"id":9,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":7,"match_type":"exact"},{"id":51,"name":"John Smith","headl...
  🔧 Calling: get_linkedin_profile({'person_id': 9})
  📋 Result: {"id":9,"name":"John Smith","headline":"Marketing Professional","city":"Singapore","country":"Singapore","industry":"Marketing","status":"student","years_experience":7,"summary":"Tax century group sim...
  🔧 Calling: search_facebook_users({'q': 'John Smith', 'limit': 5})
  📋 Result: [{"id":8,"display_name":"John Smith","city":"Kowloon","country":"Hong Kong","match_type":"exact"},{"id":51,"display_name":"John Smith","city":"Central","country":"Hong Kong","match_type":"exact"},{"id...
  🔧 Calling: get_facebook_profile({'user_id': 213})
  📋 Result: {"id":213,"display_name":"John Smith","original_nam

# Verification Reports

In [11]:
# =====================================================
#  Print detailed verification reports
# =====================================================

for cv, report in zip(all_cvs, reports):
    print("=" * 80)
    print(f"📄 Report for {cv['file']}")
    print("=" * 80)
    print(report)
    print("\n")

📄 Report for CV_1.pdf
[{'type': 'text', 'text': 'DISCREPANCIES FOUND:\n1.  **Employment Status Inconsistency (CV vs. LinkedIn):** The CV states "Engineer, ByteDance, 2020 – Present". The LinkedIn profile for John Smith (ID 9) shows the same role starting in 2020 with no end year, but the `is_current` flag is `False`. This contradicts the "Present" claim on the CV.\n2.  **Major Employment Mismatch (CV/LinkedIn vs. Facebook):** The CV and LinkedIn profile claim "Engineer, ByteDance". However, Facebook profile ID 213 (Singapore) lists "Scientist, Traveloka" as the current job, and Facebook profile ID 8 (Kowloon) lists "Engineer, Hang Seng Bank". Neither Facebook profile\'s employment details match the CV or LinkedIn.\n3.  **Major Education Level Mismatch (CV/LinkedIn vs. Facebook ID 8):** The CV and LinkedIn profile state a "Bachelor of Science (BSc) in Marketing" from McGill University. Facebook profile ID 8 (Kowloon) lists a "Doctoral Degree".\n4.  **Industry/Skill Mismatch (CV/LinkedIn

# Evaluation

In [12]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }

In [13]:
# scores is generated by the verification system above
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)

{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
